In [ ]:
#@title Prevent disconnections
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title Main Code
!apt-get -y install -qq aria2
#!pip install flask-cloudflared
!pip install requests-html

!git clone https://github.com/oobabooga/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install -r /content/text-generation-webui/extensions/openai/requirements.txt
# !mkdir repositories
# %cd /content/text-generation-webui/repositories
# !git clone https://github.com/turboderp/exllama

In [ ]:
#@title Add model

%cd /content/

from requests_html import HTMLSession
import random
import string
import secrets

OPTIONS = {
    1:"TheBloke/Mythalion-13B-GPTQ",
    2:"TheBloke/MLewdBoros-L2-13B-GPTQ",
    3:"TheBloke/MLewdBoros-LRPSGPT-2Char-13B-GPTQ",
    4:"TheBloke/Xwin-MLewd-13B-v0.2-GPTQ",
    5:"TheBloke/LLaMA2-13B-TiefighterLR-GPTQ",
    6:"TheBloke/13B-Thorns-L2-GPTQ",
    7:"TheBloke/Chronomaid-Storytelling-13B-GPTQ",
    8:"TheBloke/dolphin-2.0-mistral-7B-GPTQ",
    9:"TheBloke/Athena-v3-GPTQ",
    10:"TheBloke/LLaMA2-13B-Psyfighter2-GPTQ",
    11:"TheBloke/airoboros-l2-13B-2.2.1-GPTQ",
    12:"TheBloke/Noromaid-13B-v0.3-GPTQ",
    13:"TheBloke/SOLAR-10.7B-Instruct-v1.0-uncensored-GPTQ",
}

#@markdown ### Model presets (Uncheck the box if you use a non-preset model)
Preset = "12" #@param [1,2,3,4,5,6,7,8,9,10,11,12,13]
use_preset = True #@param {type:"boolean"}

#Black magic
CHARSET = string.ascii_letters + string.digits + string.punctuation
LENGTH = random.randint(70, 100)
Model = r"".join(secrets.choice(CHARSET) for _ in range(LENGTH))

#ADD NON-PRESET MODEL BETWEEN QUOTES
globals()[Model] = "" if use_preset == False else OPTIONS[int(Preset)]

#@markdown ### For when there are multiple large model files in the same repo. Copy/paste the url of the one you want.
large_file_url = "" #@param {type:"string"}
large_file_url = large_file_url.replace("/blob/", "/resolve/") if "/blob/" in large_file_url else large_file_url

#@markdown ### Renamed model file (change based on backend)
renamed_file = "gptq_model-4bit-128g.safetensors" #@param {type:"string"}

#Get model name
model_name = globals()[Model].split(globals()[Model][globals()[Model].rfind("/")])[-1]

#Target repo
REPO = globals()[Model] if "https://" in globals()[Model] else f"https://huggingface.co/{globals()[Model]}/tree/main?not-for-all-audiences=true"

#Get file urls
session = HTMLSession()
result = session.get(REPO)
links = result.html.absolute_links

#Create LINKS list
if len(large_file_url) > 10:
    LINKS = [large_file_url]
    for link in links:
        target = (".json" in link, ".model" in link, ".py" in link)
        if any(target) and "/resolve/" in link: LINKS.append(link)
else:
    LINKS = []
    for link in links:
        target = (".json" in link, ".model" in link, ".safetensors" in link, ".pt" in link, ".py" in link)
        if any(target) and "/resolve/" in link: LINKS.append(link)

#Name of the file containing urls for aria2
shopping_list = "down_list.txt"

#Create file for aria2 using LINKS list
with open(shopping_list, 'a') as file:
    for url in LINKS:
        target = (".safetensors" in url, ".pt" in url)
        if any(target): file.write(f"{url}\n out={renamed_file}\n")
        elif ".model" in url: file.write(f"{url}\n out=tokenizer.model\n")
        else: file.write(f"{url}\n")

#Set models destination for backend
models_folder = "/content/text-generation-webui/models"

#Download all model config files
print(f"Downloading {model_name}...\n")
!cd $models_folder && mkdir $model_name
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --summary-interval=5 --input-file=$shopping_list -d $models_folder/$model_name
!rm $shopping_list

In [ ]:
#@title LAUNCH! (Run again in case of cloudflare error)

%cd /content/text-generation-webui
!python server.py --nowebui --api --public-api --model $model_name --loader EXLLAMAV2_HF --max_seq_len 4096 --n_ctx 4096